In [118]:
library(tidyverse)
library(lubridate)
library(forecast) # Para modelos AR

In [386]:
df<-read_csv('df.csv',col_names = TRUE,cols(
  Ano = col_double(),
  Mes = col_double(),
  Ano_Mes = col_character(),
  Fecha = col_date(format = ""),
  Precios_Mensuales = col_double(),
  Aportes_Energia_GWh = col_double(),
  Volumen_Util_Diario_Energia_GWh = col_double(),
  Demanda_Energia_SIN_GWh = col_double(),
  ONI = col_double(),
  CEN_Total = col_double(),
  CEN_Hidro = col_double(),
  CEN_Termica = col_double()
))

# Creacion de funciones

In [288]:
oni<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    
    n_pronosticos<-df%>%filter(Fecha>=start_at)%>%select(ONI)%>%nrow
    
    
    while(any(is.na(df$ONI))){
        fila<-sum(complete.cases(df$ONI))+1
        m<-auto.arima(unlist(unname(df[1:fila-1,'ONI'])),max.q = 0,max.Q = 0,max.d = 0,max.D = 0)
        y_est<-round(as.vector(unlist(forecast(m, 24)['mean'])),1)
        df$ONI[is.na(df$ONI)]<-y_est
    }
    #df$ONI[df$Fecha>=as.Date(start_at)]<-round(runif(n_pronosticos,-0.5,0.5),1)
    
    return(df)
}
#ok

In [384]:
aportes<-function(df, start_at, rezagos_oni,rezagos_aportes){
    
    fechaini<-as.Date(start_at)
    #n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Aportes_Energia_GWh)%>%nrow
    #df$Aportes_Energia_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Aportes_Energia_GWh,na.rm = TRUE),max(df$Aportes_Energia_GWh,na.rm = TRUE)),3)

    #Definir el dataframe Aportes, ONI

    temp<-df%>%select(Fecha, Aportes_Energia_GWh, ONI, Aportes_Normalizados)
    #rezagos_oni<-12                                                          #Rezagos para el ONI (parametrizable)
    #rezagos_aportes<-24                                                      #Rezagos para los Aportes (parametrizable)

        for(j in seq(rezagos_oni)){                                             #Creacion de columnas rezagadas ONI
            col_oni<-paste0('ONI_',j)                                           #   
            temp[[col_oni]]<-lag(df$ONI,j)                                      #
        }                                                                       #

        for(k in seq(rezagos_aportes)){                                         #Creacion de columnas rezagadas Aportes
            col_aportes<-paste0('Aportes_',k)                                   #
            temp[[col_aportes]]<-lag(df$Aportes_Normalizados,k)                 #
        }                                                                       #


        #new_data<-temp%>%filter(Fecha>=as.Date('2019-06-01')%m+%months(-24)

        temp<-temp%>%select(-1,-2,-3,Aportes_Normalizados,                      #Elimina Fecha, Aportes_Energia, ONI
                                    starts_with('Aportes_[0-9]'),               #Trae los rezagos de Aportes
                                    starts_with('ONI_'))            #Trae los rezagos de ONI


        new_data<-temp

        temp<-temp%>%drop_na()

        #Aplicamos un modelo Forward Selection
        m_FitAll<-lm(Aportes_Normalizados~.,data = temp)
        m<-(step(lm(Aportes_Normalizados~1,data=temp),direction = 'forward',scope = formula(m_FitAll),trace = 0))

        # Seleccionar solo variables significativas
        toselect.x<-summary(m)$coeff[,4] < 0.05
        relevant.x <- names(toselect.x)[toselect.x == TRUE] 
        sig.formula <- as.formula(paste("Aportes_Normalizados ~",paste(relevant.x,collapse = '+'))) 

        m<-lm(formula = sig.formula,data = temp)


        while(df$Aportes_Normalizados[!complete.cases(df$Aportes_Normalizados)]%>%length>0){
            df[!complete.cases(df),'Aportes_Normalizados'][[1]][1]<-unlist(unname(forecast(object = m, h = 1,newdata = new_data%>%tail(25)%>%head(1)%>%select(-1,everything()))['mean']))

            new_data<-new_data%>%drop_na()
        }
    return(df)
}

In [290]:
reservas<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Volumen_Util_Diario_Energia_GWh)%>%nrow
    df$Volumen_Util_Diario_Energia_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Volumen_Util_Diario_Energia_GWh,na.rm = TRUE),max(df$Volumen_Util_Diario_Energia_GWh,na.rm = TRUE)),3)
    
    return(df)
}

In [291]:
precios<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Precios_Mensuales)%>%nrow
    df$Precios_Mensuales[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Precios_Mensuales,na.rm = TRUE),max(df$Precios_Mensuales,na.rm = TRUE)),3)
    
    return(df)
}

In [292]:
demanda<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Demanda_Energia_SIN_GWh)%>%nrow
    df$Demanda_Energia_SIN_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Demanda_Energia_SIN_GWh,na.rm = TRUE),max(df$Demanda_Energia_SIN_GWh,na.rm = TRUE)),3)
    
    return(df)
}

In [293]:
columnas_calculadas<-function(df){
    #debo montar dem_CEN
    df$Dem_CEN<-round(df$Demanda_Energia_SIN_GWh/(as.vector(days_in_month(df$Fecha))*df$CEN_Total*24),4)
    #debo montar Term_Hidro
    df$Termica_Hidraulica<-round(df$CEN_Termica/df$CEN_Hidro,4)
    
    return(df)
}

In [294]:
normalizar_hidrologia<-function(df){
    df<-df%>%
        left_join(
                df%>%arrange(Mes)%>%group_by(Mes)%>%summarize(mean_aportes = mean(Aportes_Energia_GWh,na.rm = TRUE), 
                                                              mean_reservas = mean(Volumen_Util_Diario_Energia_GWh,na.rm = TRUE),
                                                              sd_aportes = sd(Aportes_Energia_GWh, na.rm = TRUE),
                                                              sd_reservas = sd(Volumen_Util_Diario_Energia_GWh,na.rm = TRUE)),by='Mes')%>%
    mutate(
        Aportes_Normalizados = (Aportes_Energia_GWh-mean_aportes)/sd_aportes,
        Reservas_Normalizadas = (Volumen_Util_Diario_Energia_GWh-mean_reservas)/sd_reservas)
    
    return(df)
}

# Probar funciones

In [ ]:
df<-normalizar_hidrologia(df)
#df<-precios(df,start_at = '2019-06-01')
df<-oni(df,start_at = '2019-06-01')
df<-aportes(df,start_at = '2019-06-01',rezagos_oni = 12,rezagos_aportes = 24)
# df<-reservas(df,start_at = '2019-06-01')
#df<-demanda(df,start_at = '2019-06-01')
#df<-columnas_calculadas(df)


In [383]:
#Definir el dataframe Aportes, ONI
temp<-df%>%select(Fecha, Aportes_Energia_GWh, ONI, Aportes_Normalizados)
#for(fecha in seq.Date(as.Date('2019-06-01'), to=as.Date('2020-01-01'), by = 'month')){

    rezagos_oni<-12                                                          #Rezagos para el ONI (parametrizable)
    rezagos_aportes<-24                                                      #Rezagos para los Aportes (parametrizable)

        for(j in seq(rezagos_oni)){                                             #Creacion de columnas rezagadas ONI
            col_oni<-paste0('ONI_',j)                                           #   
            temp[[col_oni]]<-lag(df$ONI,j)                                      #
        }                                                                       #

        for(k in seq(rezagos_aportes)){                                         #Creacion de columnas rezagadas Aportes
            col_aportes<-paste0('Aportes_',k)                                   #
            temp[[col_aportes]]<-lag(df$Aportes_Normalizados,k)                 #
        }                                                                       #


        #new_data<-temp%>%filter(Fecha>=as.Date('2019-06-01')%m+%months(-24)

        temp<-temp%>%select(-1,-2,-3,Aportes_Normalizados,                      #Elimina Fecha, Aportes_Energia, ONI
                                    starts_with('Aportes_[0-9]'),               #Trae los rezagos de Aportes
                                    starts_with('ONI_'))            #Trae los rezagos de ONI


        new_data<-temp

        temp<-temp%>%drop_na()

        #Aplicamos un modelo Forward Selection
        m_FitAll<-lm(Aportes_Normalizados~.,data = temp)
        m<-(step(lm(Aportes_Normalizados~1,data=temp),direction = 'forward',scope = formula(m_FitAll),trace = 0))

        # Seleccionar solo variables significativas
        toselect.x<-summary(m)$coeff[,4] < 0.01
        relevant.x <- names(toselect.x)[toselect.x == TRUE] 
        sig.formula <- as.formula(paste("Aportes_Normalizados ~",paste(relevant.x,collapse = '+'))) 

        m<-lm(formula = sig.formula,data = temp)


        while(df$Aportes_Normalizados[!complete.cases(df$Aportes_Normalizados)]%>%length>0){
            df[!complete.cases(df),'Aportes_Normalizados'][[1]][1]<-unlist(unname(forecast(object = m, h = 1,newdata = new_data%>%tail(25)%>%head(1)%>%select(-1,everything()))['mean']))

            new_data<-new_data%>%drop_na()
        }
return(df)

Ano,Mes,Ano_Mes,Fecha,Precios_Mensuales,Aportes_Energia_GWh,Volumen_Util_Diario_Energia_GWh,Demanda_Energia_SIN_GWh,ONI,CEN_Total,CEN_Hidro,CEN_Termica,mean_aportes,mean_reservas,sd_aportes,sd_reservas,Aportes_Normalizados,Reservas_Normalizadas
2000,1,2000-01,2000-01-01,36.77878,1900.660,10115.866,3354.237,-1.7,11.88597,7.97497,3.897,2387.357,10418.906,860.5643,1018.8277,-0.56555556,-0.29743960
2000,2,2000-02,2000-02-01,40.26148,1993.636,9705.065,3278.267,-1.4,11.88597,7.97497,3.897,1977.469,8931.573,571.9035,947.0586,0.02826836,0.81673089
2000,3,2000-03,2000-03-01,37.63730,2253.361,9222.966,3495.203,-1.1,11.96897,8.05797,3.897,2921.435,7971.466,1105.0457,1191.9835,-0.60456759,1.04993055
2000,4,2000-04,2000-04-01,44.29806,2429.651,8945.684,3298.386,-0.8,12.05397,8.14297,3.897,4456.870,8298.807,1832.5230,1671.8904,-1.10624476,0.38691375
2000,5,2000-05,2000-05-01,37.29112,4817.901,10131.148,3502.127,-0.7,12.29927,8.23427,4.051,6027.007,9605.625,1735.7382,1865.8465,-0.69659476,0.28165408
2000,6,2000-06,2000-06-01,39.99320,4889.563,11020.110,3367.886,-0.6,12.44327,8.23427,4.195,5812.246,10814.714,1105.3876,1832.7577,-0.83471448,0.11206968
2000,7,2000-07,2000-07-01,42.80114,4485.164,11302.052,3459.363,-0.6,12.44927,8.24027,4.195,5356.389,11468.071,1090.3524,1628.1093,-0.79903026,-0.10197044
2000,8,2000-08,2000-08-01,49.07017,4450.365,11371.624,3518.519,-0.5,12.44927,8.24027,4.195,4724.812,11611.741,645.6103,1303.1425,-0.42509697,-0.18426008
2000,9,2000-09,2000-09-01,59.53525,4497.103,11396.880,3456.203,-0.5,12.58267,8.26467,4.304,4098.956,11522.656,758.1609,1099.4048,0.52514913,-0.11440373
2000,10,2000-10,2000-10-01,48.42079,3751.008,11357.160,3576.068,-0.6,12.58267,8.26467,4.304,4548.208,11791.016,1034.7088,1117.0774,-0.77045824,-0.38838448


In [355]:
while(df$Aportes_Normalizados[!complete.cases(df$Aportes_Normalizados)]%>%length>0){
    # Este es siguiente valor de Aportes_Norma
    df[!complete.cases(df),'Aportes_Normalizados'][[1]][1]<-unlist(unname(forecast(object = m, h = 1,newdata = new_data%>%tail(25)%>%head(1)%>%select(-1,everything()))['mean']))
}

In [358]:
df%>%filter(Fecha=='2019-07-01')

Ano,Mes,Ano_Mes,Fecha,Precios_Mensuales,Aportes_Energia_GWh,Volumen_Util_Diario_Energia_GWh,Demanda_Energia_SIN_GWh,ONI,CEN_Total,CEN_Hidro,CEN_Termica,mean_aportes,mean_reservas,sd_aportes,sd_reservas,Aportes_Normalizados,Reservas_Normalizadas
2019,7,2019-07,2019-07-01,NA,NA,NA,NA,0.7,11.88597,7.97497,3.897,5356.389,11468.07,1090.352,1628.109,NA,NA


[1] 23

In [333]:
new_data%>%tail(25)%>%head(1)%>%select(everything())




Aportes_Normalizados,ONI_1,ONI_2,ONI_3,ONI_4,ONI_5,ONI_6,ONI_7,ONI_8,ONI_9,⋯,Aportes_15,Aportes_16,Aportes_17,Aportes_18,Aportes_19,Aportes_20,Aportes_21,Aportes_22,Aportes_23,Aportes_24
0.5570397,0.8,0.8,0.8,0.8,0.8,0.9,0.7,0.4,0.2,⋯,0.574852,1.521532,0.2113782,0.4793851,0.03917255,1.04827,0.1151529,0.7846522,1.290809,1.635569


In [324]:
new_data[new_data[complete.cases(new_data),]%>%nrow+1,]

Aportes_Normalizados,ONI_1,ONI_2,ONI_3,ONI_4,ONI_5,ONI_6,ONI_7,ONI_8,ONI_9,⋯,Aportes_15,Aportes_16,Aportes_17,Aportes_18,Aportes_19,Aportes_20,Aportes_21,Aportes_22,Aportes_23,Aportes_24
1.290809,0.4,0.3,0.1,-0.1,-0.3,-0.6,-0.7,-0.7,-0.7,⋯,-0.8704176,-0.7207485,-0.9197372,-0.9576918,-0.7818678,-1.437234,-0.9034445,1.19706,0.7543429,1.061577


In [347]:
df[!complete.cases(df),'Aportes_Normalizados'][[1]][1]

[1] NA

In [299]:
#temp%>%filter(Fecha==as.Date('2019-06-01')%m+%months(-24)

In [285]:
temp

Aportes_Normalizados,ONI_1,ONI_2,ONI_3,ONI_4,ONI_5,ONI_6,ONI_7,ONI_8,ONI_9,⋯,Aportes_15,Aportes_16,Aportes_17,Aportes_18,Aportes_19,Aportes_20,Aportes_21,Aportes_22,Aportes_23,Aportes_24
-0.7762716,-0.3,-0.3,-0.3,-0.2,-0.1,-0.1,-0.1,-0.3,-0.3,⋯,-0.7704582,0.5251491,-0.4250970,-0.7990303,-0.8347145,-0.6965948,-1.1062448,-0.6045676,0.02826836,-0.56555556
-1.2193980,-0.1,-0.3,-0.3,-0.3,-0.2,-0.1,-0.1,-0.1,-0.3,⋯,-1.1530193,-0.7704582,0.5251491,-0.4250970,-0.7990303,-0.8347145,-0.6965948,-1.1062448,-0.60456759,0.02826836
-0.7485983,0.0,-0.1,-0.3,-0.3,-0.3,-0.2,-0.1,-0.1,-0.1,⋯,-1.0795624,-1.1530193,-0.7704582,0.5251491,-0.4250970,-0.7990303,-0.8347145,-0.6965948,-1.10624476,-0.60456759
-0.2487314,0.1,0.0,-0.1,-0.3,-0.3,-0.3,-0.2,-0.1,-0.1,⋯,-1.0083713,-1.0795624,-1.1530193,-0.7704582,0.5251491,-0.4250970,-0.7990303,-0.8347145,-0.69659476,-1.10624476
-0.7247611,0.2,0.1,0.0,-0.1,-0.3,-0.3,-0.3,-0.2,-0.1,⋯,-1.4293733,-1.0083713,-1.0795624,-1.1530193,-0.7704582,0.5251491,-0.4250970,-0.7990303,-0.83471448,-0.69659476
0.4589239,0.4,0.2,0.1,0.0,-0.1,-0.3,-0.3,-0.3,-0.2,⋯,-0.8954268,-1.4293733,-1.0083713,-1.0795624,-1.1530193,-0.7704582,0.5251491,-0.4250970,-0.79903026,-0.83471448
-0.5579440,0.7,0.4,0.2,0.1,0.0,-0.1,-0.3,-0.3,-0.3,⋯,-1.3490221,-0.8954268,-1.4293733,-1.0083713,-1.0795624,-1.1530193,-0.7704582,0.5251491,-0.42509697,-0.79903026
-0.3433351,0.8,0.7,0.4,0.2,0.1,0.0,-0.1,-0.3,-0.3,⋯,-1.3427297,-1.3490221,-0.8954268,-1.4293733,-1.0083713,-1.0795624,-1.1530193,-0.7704582,0.52514913,-0.42509697
-1.3041126,0.9,0.8,0.7,0.4,0.2,0.1,0.0,-0.1,-0.3,⋯,-1.5185992,-1.3427297,-1.3490221,-0.8954268,-1.4293733,-1.0083713,-1.0795624,-1.1530193,-0.77045824,0.52514913
-1.5551348,1.0,0.9,0.8,0.7,0.4,0.2,0.1,0.0,-0.1,⋯,-1.4990317,-1.5185992,-1.3427297,-1.3490221,-0.8954268,-1.4293733,-1.0083713,-1.0795624,-1.15301928,-0.77045824


In [270]:
m


Call:
lm(formula = sig.formula, data = temp)

Coefficients:
(Intercept)    Aportes_1        ONI_1    Aportes_6       ONI_12  
    0.06106      0.43466     -0.32206      0.22913      0.14899  


In [231]:
with(temp, predict(object = sig.formula, h = 1))

Warning message in mean.default(x, na.rm = TRUE):
“argument is not numeric or logical: returning NA”

ERROR: Error in x - fits: non-numeric argument to binary operator


In [122]:
temp%>%drop_na()%>%head

Fecha,Aportes_Energia_GWh,ONI,Aportes_Normalizados,ONI_1,ONI_2,ONI_3,ONI_4,ONI_5,ONI_6,⋯,Aportes_15,Aportes_16,Aportes_17,Aportes_18,Aportes_19,Aportes_20,Aportes_21,Aportes_22,Aportes_23,Aportes_24
2002-01-01,1719.325,-0.1,-0.7762716,-0.3,-0.3,-0.3,-0.2,-0.1,-0.1,⋯,-0.7704582,0.5251491,-0.4250970,-0.7990303,-0.8347145,-0.6965948,-1.1062448,-0.6045676,0.02826836,-0.56555556
2002-02-01,1280.091,0.0,-1.2193980,-0.1,-0.3,-0.3,-0.3,-0.2,-0.1,⋯,-1.1530193,-0.7704582,0.5251491,-0.4250970,-0.7990303,-0.8347145,-0.6965948,-1.1062448,-0.60456759,0.02826836
2002-03-01,2094.200,0.1,-0.7485983,0.0,-0.1,-0.3,-0.3,-0.3,-0.2,⋯,-1.0795624,-1.1530193,-0.7704582,0.5251491,-0.4250970,-0.7990303,-0.8347145,-0.6965948,-1.10624476,-0.60456759
2002-04-01,4001.064,0.2,-0.2487314,0.1,0.0,-0.1,-0.3,-0.3,-0.3,⋯,-1.0083713,-1.0795624,-1.1530193,-0.7704582,0.5251491,-0.4250970,-0.7990303,-0.8347145,-0.69659476,-1.10624476
2002-05-01,4769.012,0.4,-0.7247611,0.2,0.1,0.0,-0.1,-0.3,-0.3,⋯,-1.4293733,-1.0083713,-1.0795624,-1.1530193,-0.7704582,0.5251491,-0.4250970,-0.7990303,-0.83471448,-0.69659476
2002-06-01,6319.534,0.7,0.4589239,0.4,0.2,0.1,0.0,-0.1,-0.3,⋯,-0.8954268,-1.4293733,-1.0083713,-1.0795624,-1.1530193,-0.7704582,0.5251491,-0.4250970,-0.79903026,-0.83471448


In [547]:
generar_archivos<-function(df, start_at, n_iter, folder_location){
    
    
    if(dir.exists(folder_location)){
        while(file.remove(list.files(path = folder_location,pattern = '[0-9][0-9][0-9][0-9][0-9].csv',full.names = TRUE)[1])==TRUE){
            list.files(path = folder_location,pattern = '[0-9][0-9][0-9][0-9][0-9].csv',full.names = TRUE)[1]
        }

        fechaini<-as.Date(start_at)

        for( i in seq(n_iter)){

            #Normalizar aportes
            #normalizar reservas
            #Agregar variables calculadas
            df<-precios(df,fechaini)
            df<-oni(df,fechaini)
            df<-aportes(df,fechaini)
            df<-reservas(df,fechaini)
            df<-demanda(df,fechaini)
            df<-columnas_calculadas(df)

            #pronostico de variables calculadas
            write_csv(df, paste0(folder_location,str_pad(i, pad = 0, width = 5),'.csv'))
        }
        cat(paste0(n_iter, ' archivos generados en la ruta ',folder_location))
    } else{
        cat('Error: Directorio no existe en esta máquina')
    }

}

In [548]:
generar_archivos(df,start_at = '2019-06-01',n_iter = 20,folder_location = './Resultados/')

20 archivos generados en la ruta ./Resultados/

# Tareas

In [235]:
#hacer el arx pronosticar uno a uno y tiene como entrada el ONI y se devuelven las unidades 2h
#hacer arx para reservas con oni, aportes, relacion T/H y me devuelvo 2h
#un modelo simple de precio 2h